# Importing libraries

In [2]:
import os
import warnings
import numpy as np
import pandas as pd
warnings.filterwarnings('ignore')

# Data visualisation libraries                                                 
import seaborn as sns
import matplotlib.pyplot as plt             # for plotting graphs
from matplotlib import rcParams             # for validating 
from folium.plugins import MarkerCluster    # for visualising clusters

%matplotlib inline

In [3]:
import datetime
datetime.datetime.strptime

<function datetime.strptime>

# Importing dataset 

In [187]:
food_metadata = pd.read_csv('epi_r.csv')
food_metadata.shape

(20052, 680)

In [188]:
food_metadata.tail()

,title,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
20047,Parmesan Puffs,3.125,28.0,2.0,2.0,64.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20048,Artichoke and Parmesan Risotto,4.375,671.0,22.0,28.0,583.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20049,Turkey Cream Puff Pie,4.375,563.0,31.0,38.0,652.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
20050,Snapper on Angel Hair with Citrus Cream,4.375,631.0,45.0,24.0,517.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20051,Baked Ham with Marmalade-Horseradish Glaze,4.375,560.0,73.0,10.0,3698.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


dropping rows with null values from the following columns

In [189]:
food_metadata = food_metadata.dropna(subset=['calories', 'protein', 'fat', 'sodium'])
dftemp = food_metadata[['title', 'rating', 'calories', 'protein', 'fat', 'sodium']]
dftemp.shape

(15864, 6)

In [190]:
dftemp.isna().sum()

title       0
rating      0
calories    0
protein     0
fat         0
sodium      0
dtype: int64

In [191]:
dftemp.head()

,title,rating,calories,protein,fat,sodium
0,"Lentil, Apple, and Turkey Wrap",2.500,426.0,30.0,7.0,559.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.0,18.0,23.0,1439.0
2,Potato and Fennel Soup Hodge,3.750,165.0,6.0,7.0,165.0
4,Spinach Noodle Casserole,3.125,547.0,20.0,32.0,452.0
5,The Best Blts,4.375,948.0,19.0,79.0,1042.0


In [192]:
dftemp.describe()

,rating,calories,protein,fat,sodium
count,15864.000000,1.586400e+04,15864.000000,1.586400e+04,1.586400e+04
mean,3.760952,6.350683e+03,100.324571,3.469868e+02,6.252742e+03
std,1.285518,3.598484e+05,3843.462312,2.045933e+04,3.340421e+05
min,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00
25%,3.750000,1.990000e+02,3.000000,7.000000e+00,8.200000e+01
50%,4.375000,3.330000e+02,8.000000,1.700000e+01,2.960000e+02
75%,4.375000,5.872500e+02,27.000000,3.300000e+01,7.130000e+02
max,5.000000,3.011122e+07,236489.000000,1.722763e+06,2.767511e+07


In [193]:
food_metadata = dftemp

In [195]:
food_metadata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15864 entries, 0 to 20051
Columns: 680 entries, title to turkey
dtypes: float64(679), object(1)
memory usage: 82.4+ MB


In [196]:
#df2 = pd.DataFrame({'Protein': food_metadata['protein']}, index=food_metadata['title'])
#ax = df2.plot.bar(rot=0)

In [199]:
food_metadata.sample(2)

,title,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
14922,Grilled Chicken and Vegetable Sandwiches with ...,4.375,500.0,14.0,30.0,659.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4278,Candied Cashews,5.000,230.0,6.0,18.0,124.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [197]:
descriptions = food_metadata['calories'].apply(str) +' '+ food_metadata['protein'].apply(str) + ' ' + food_metadata['fat'].apply(str) +' '+ food_metadata['sodium'].apply(str)

# Printing the first description
descriptions[600]

'261.0 3.0 15.0 119.0'

Creating a similarity matrix between the food items

In [172]:
# import TfidfVector from sklearn.

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

def create_similarity_matrix(new_description, overall_descriptions):
#Append the new description to the overall set.
        overall_descriptions.append(new_description)
        # Define a tfidf vectorizer and remove all stopwords.
        tfidf = TfidfVectorizer(stop_words = "english")
        #Convert tfidf matrix by fitting and transforming the data.
        tfidf_matrix = tfidf.fit_transform(overall_descriptions)
        # output the shape of the matrix.
        tfidf_matrix.shape
        # calculating the cosine similarity matrix.
        cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
        
        return cosine_sim

defining the get_recommendations() function

In [173]:
def get_recommendations(new_description,overall_descriptions):
    # create the similarity matrix
    cosine_sim = create_similarity_matrix(new_description, overall_descriptions)
    # Get pairwise similarity scores of all the students with new student.
    sim_scores = list(enumerate(cosine_sim[-1]))
    # Sort the descriptions based on similarity score.
    sim_scores = sorted(sim_scores, key = lambda x:x[1], reverse = True )
    # Get the scores of top 10 descriptions.
    sim_scores = sim_scores[1:10]
    # Get the student indices.
    indices = [i[0]for i in sim_scores]
    
    return df.iloc[indices]

In [174]:
# Recommend the food to a new user.

new_description = pd.Series('28.0 2.0 2.0 6.0')
output = get_recommendations(new_description, descriptions)

In [175]:
output = output.sort_values(by=['rating'], ascending=False) #sorting the recommendations by rating

### Final List of Recommendations (output)

In [176]:
output['title'] #

9522      Maple-Glazed Parsnips with Popped Mustard Seeds 
15105                                Miso Glazed Sea Bass 
6531     Spaghetti with Sweet 100 Tomatoes, Garlic Chiv...
8102            Curried Parsnip Soup with Shredded Apples 
104                                          Stout Floats 
10332    Rum Cake with Rum Raisin Ice Cream and Island ...
11516                                Chile-Infused Aperol 
16635                                Chile-Infused Aperol 
14705    Lemon Pudding Filled Coconut Cupcakes with Sha...
Name: title, dtype: object

### Exporting model using keras

In [177]:
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model
warnings.filterwarnings('ignore')
%matplotlib inline

In [178]:
df.head()

,title,rating,calories,protein,fat,sodium
0,"Lentil, Apple, and Turkey Wrap",2.500,426.0,30.0,7.0,559.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.0,18.0,23.0,1439.0
2,Potato and Fennel Soup Hodge,3.750,165.0,6.0,7.0,165.0
4,Spinach Noodle Casserole,3.125,547.0,20.0,32.0,452.0
5,The Best Blts,4.375,948.0,19.0,79.0,1042.0


In [179]:
train, test = train_test_split(df, test_size=0.2, random_state=42)

In [180]:
n_cal = len(df['calories'].unique())
n_protein = len(df['protein'].unique())
n_fat = len(df['fat'].unique())
n_sodium = len(df['sodium'].unique())

In [181]:
print(n_cal)
print(n_protein)
print(n_fat)
print(n_sodium)

1858
282
326
2433


In [182]:
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense
from tensorflow.keras.models import Model

calories_input = Input(shape=[1], name="Calorie-Input")
calories_embedding = Embedding(n_cal+1, 5, name="Calorie-Embedding")(calories_input)
calories_vec = Flatten(name="Flatten-Calorie")(calories_embedding)

pro_input = Input(shape=[1], name="Protein-Input")
pro_embedding = Embedding(n_protein+1, 5, name="Protein-Embedding")(pro_input)
pro_vec = Flatten(name="Flatten-Protein")(pro_embedding)

fat_input = Input(shape=[1], name="fat-Input")
fat_embedding = Embedding(n_fat+1, 5, name="fat-Embedding")(fat_input)
fat_vec = Flatten(name="Flatten-fat")(fat_embedding)

sod_input = Input(shape=[1], name="sodium-Input")
sod_embedding = Embedding(n_sodium+1, 5, name="sodium-Embedding")(sod_input)
sod_vec = Flatten(name="Flatten-sodium")(sod_embedding)

# concatenate features
conc = Concatenate()([calories_vec, pro_vec, fat_vec, sod_vec])

# add fully-connected-layers
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
out = Dense(1)(fc2)

# Create model and compile it
model = Model([calories_input, pro_input, fat_input, sod_input], out)
model.compile('adam', 'mean_squared_error')
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Calorie-Input (InputLayer)      [(None, 1)]          0                                            
__________________________________________________________________________________________________
Protein-Input (InputLayer)      [(None, 1)]          0                                            
__________________________________________________________________________________________________
fat-Input (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
sodium-Input (InputLayer)       [(None, 1)]          0                                            
____________________________________________________________________________________________

In [1]:
# Fittiing the model and saving it

#history = model.fit([train.calories, train.protein, train.fat, train.sodium], train.rating, epochs=10, verbose=1)
#model.save('meals_model.h5')